# Open AI mudelite peenhäälestamine

See märkmik põhineb Open AI [Peenhäälestamise](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dokumentatsioonis esitatud juhistel.

Peenhäälestamine parandab baasmudelite jõudlust teie rakenduse jaoks, treenides mudelit uuesti täiendavate andmete ja kontekstiga, mis on seotud konkreetse kasutusjuhtumi või stsenaariumiga. Pange tähele, et viipade kujundamise tehnikad, nagu _few shot learning_ ja _retrieval augmented generation_, võimaldavad teil täiustada vaikimisi viipa asjakohaste andmetega, et parandada kvaliteeti. Kuid nende lähenemisviiside piiranguks on sihtbaasmudeli maksimaalne tokenite akna suurus.

Peenhäälestamise abil treenime mudelit ise uuesti vajalike andmetega (mis võimaldab meil kasutada palju rohkem näiteid, kui mahub maksimaalsesse tokenite aknasse) ja juurutame _kohandatud_ versiooni mudelist, mis ei vaja enam näiteid järeldamise ajal. See mitte ainult ei paranda meie viipade kujundamise tõhusust (meil on rohkem paindlikkust tokenite akna kasutamisel muudel eesmärkidel), vaid võib potentsiaalselt ka kulusid vähendada (vähendades tokenite arvu, mida peame mudelile järeldamise ajal saatma).

Peenhäälestamine koosneb neljast etapist:
1. Valmistage ette treeningandmed ja laadige need üles.
1. Käivitage treeningprotsess, et saada peenhäälestatud mudel.
1. Hinnake peenhäälestatud mudelit ja tehke kvaliteedi parandamiseks iteratsioone.
1. Juurutage peenhäälestatud mudel järeldamiseks, kui olete rahul.

Pange tähele, et mitte kõik baasmudelid ei toeta peenhäälestamist - [vaadake OpenAI dokumentatsiooni](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) uusima teabe saamiseks. Samuti on võimalik peenhäälestada varem peenhäälestatud mudelit. Selles juhendis kasutame peenhäälestamise sihtbaasmudelina `gpt-35-turbo`.

---


### Samm 1.1: Valmista oma andmestik

Loome vestlusroboti, mis aitab sul mõista keemiliste elementide perioodilisustabelit, vastates küsimustele iga elemendi kohta limeriku vormis. Selles _lihtsas_ õpetuses loome andmestiku, et treenida mudelit mõne näidisvastusega, mis näitavad andmete oodatavat vormingut. Päriselu kasutusjuhtumites oleks vaja koostada palju rohkemate näidetega andmestik. Võimalik, et saad kasutada ka avatud andmestikku (oma rakenduse valdkonnas), kui see on olemas, ja vormindada see peenhäälestuseks sobivaks.

Kuna keskendume `gpt-35-turbo` mudelile ja otsime ühe pöörde vastust (vestluse lõpetamist), saame luua näiteid, kasutades [seda soovitatud vormingut](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), mis vastab OpenAI vestluse lõpetamise nõuetele. Kui eeldad mitme pöördega vestluse sisu, kasutaksid [mitme pöörde näite vormingut](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), mis sisaldab `weight` parameetrit, et näidata, milliseid sõnumeid tuleks (või ei tuleks) peenhäälestuse protsessis kasutada.

Siin õpetuses kasutame lihtsamat ühe pöörde vormingut. Andmed on [jsonl-vormingus](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst), kus iga kirje on esitatud ühe rea kohta, igaüks JSON-vormingus objektina. Allolev koodinäide näitab 2 kirjet näidisena – vaata [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl), et näha täielikku näidiskomplekti (10 näidet), mida kasutame oma peenhäälestuse õpetuses. **Märkus:** Iga kirje _peab_ olema määratletud ühel real (mitte jagatud mitmele reale, nagu tavaliselt vormindatud JSON-failides).

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

Päriselu kasutusjuhtumites on vaja palju suuremat näidete komplekti, et saavutada häid tulemusi – kompromiss seisneb vastuste kvaliteedi ja peenhäälestuse aja/kulude vahel. Kasutame väikest komplekti, et saaksime peenhäälestuse kiiresti lõpule viia ja protsessi illustreerida. Vaata [seda OpenAI Cookbook näidet](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst), et näha keerukamat peenhäälestuse õpetust.


---

### Samm 1.2 Laadi oma andmestik üles

Laadi andmed üles, kasutades Files API-d [nagu siin kirjeldatud](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Pange tähele, et selle koodi käivitamiseks peate esmalt tegema järgmised sammud:
 - Installima `openai` Python paketi (veenduge, et kasutate versiooni >=0.28.0, et saada uusimad funktsioonid)
 - Määrama `OPENAI_API_KEY` keskkonnamuutuja oma OpenAI API võtmega
Lisateabe saamiseks vaadake kursuse jaoks pakutud [Seadistamise juhendit](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst).

Nüüd käivitage kood, et luua fail üleslaadimiseks oma kohalikust JSONL-failist.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### Samm 2.1: Loo peenhäälestamise töö SDK abil

Peenhäälestamise töö loomiseks kasutage SDK-d, mis võimaldab teil määrata vajalikud parameetrid ja algatada protsessi. Veenduge, et kõik vajalikud eeldused oleksid täidetud, näiteks andmekogumite ettevalmistamine ja õigete konfiguratsioonide seadistamine.

#### Näide: Peenhäälestamise töö loomine

Allpool on näide, kuidas peenhäälestamise töö luua, kasutades SDK-d. Kohandage koodi vastavalt oma vajadustele:

```python
# Importige vajalikud moodulid
from sdk_module import FineTuneJob

# Määrake peenhäälestamise töö parameetrid
job_parameters = {
    "training_data": "path/to/training_data.json",
    "model": "base_model_name",
    "epochs": 5,
    "batch_size": 32
}

# Loo peenhäälestamise töö
fine_tune_job = FineTuneJob.create(job_parameters)

# Käivitage töö
fine_tune_job.start()
```

#### Olulised märkused:
- Veenduge, et `training_data` viitab õigesti ettevalmistatud andmekogumile.
- Kohandage `epochs` ja `batch_size` vastavalt oma mudeli ja ressursinõuetele.
- Kontrollige dokumentatsiooni, et saada täpsemat teavet SDK funktsioonide kohta.

Kui töö on käivitatud, saate jälgida selle edenemist ja tulemusi SDK kaudu või vastava kasutajaliidese kaudu.


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Samm 2.2: Kontrolli töö olekut

Siin on mõned asjad, mida saab teha `client.fine_tuning.jobs` API-ga:
- `client.fine_tuning.jobs.list(limit=<n>)` - Kuvab viimased n peenhäälestuse tööd
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Annab konkreetse peenhäälestuse töö üksikasjad
- `client.fine_tuning.jobs.cancel(<job_id>)` - Tühistab peenhäälestuse töö
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Kuvab kuni n sündmust töö kohta
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Protsessi esimene samm on _treeningfaili valideerimine_, et veenduda, et andmed on õiges vormingus.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### Samm 2.3: Jälgi sündmusi, et jälgida edusamme


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Samm 2.4: Vaata staatust OpenAI juhtpaneelil


Saate vaadata olekut ka OpenAI veebilehte külastades ja platvormi _Fine-tuning_ sektsiooni uurides. Seal kuvatakse praeguse töö olek ning võimaldatakse jälgida varasemate tööde täitmise ajalugu. Sellel ekraanipildil näete, et eelmine täitmine ebaõnnestus, kuid teine katse õnnestus. Konteksti selgitamiseks: esimene katse ebaõnnestus, kuna kasutati valesti vormindatud kirjetega JSON-faili – pärast parandamist teine katse lõpetati edukalt ja mudel tehti kasutamiseks kättesaadavaks.

![Fine-tuning töö olek](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.et.png)


Saate vaadata olekuteateid ja mõõdikuid, kerides visuaalsel juhtpaneelil allapoole, nagu näidatud:

| Sõnumid | Mõõdikud |
|:---|:---|
| ![Sõnumid](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.et.png) |  ![Mõõdikud](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.et.png)|


---

### Samm 3.1: ID hankimine ja peenhäälestatud mudeli testimine koodis


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Samm 3.2: Laadi ja testi peenhäälestatud mudelit Playgroundis

Nüüd saate testida peenhäälestatud mudelit kahel viisil. Esiteks võite külastada Playgroundi ja valida mudelite rippmenüüst oma äsja peenhäälestatud mudeli. Teine võimalus on kasutada peenhäälestamise paneelil kuvatavat "Playground" valikut (vt ülaltoodud ekraanipilti), mis avab järgmise _võrdleva_ vaate, kus on kõrvuti näha algmudel ja peenhäälestatud mudel kiireks hindamiseks.

![Peenhäälestamise töö olek](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.et.png)

Lihtsalt täitke süsteemi kontekst, mida kasutasite oma treeningandmetes, ja esitage oma testküsimus. Te märkate, et mõlemad pooled uuendatakse identse konteksti ja küsimusega. Käivitage võrdlus ja näete erinevust nende väljundites. _Pange tähele, kuidas peenhäälestatud mudel esitab vastuse teie näidetes antud formaadis, samas kui algmudel järgib lihtsalt süsteemi juhist_.

![Peenhäälestamise töö olek](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.et.png)

Te märkate, et võrdlus näitab ka iga mudeli tokenite arvu ja järelduse tegemiseks kulunud aega. **See konkreetne näide on lihtsustatud, et näidata protsessi, kuid ei kajasta tegelikku andmekogumit ega stsenaariumi**. Võite märgata, et mõlemad näited näitavad sama arvu tokeneid (süsteemi kontekst ja kasutaja küsimus on identsed), kuid peenhäälestatud mudelil kulub järelduse tegemiseks rohkem aega (kohandatud mudel).

Reaalsetes olukordades ei kasuta te sellist lihtsat näidet, vaid peenhäälestate mudelit päris andmete (nt tootekataloog klienditeeninduseks) põhjal, kus vastuse kvaliteet on palju ilmsem. _Sellises_ kontekstis nõuab algmudeliga samaväärse vastuse kvaliteedi saavutamine rohkem kohandatud juhiste koostamist, mis suurendab tokenite kasutust ja potentsiaalselt ka järelduse töötlemiseks kuluvat aega. _Selle proovimiseks vaadake OpenAI Cookbook'i peenhäälestamise näiteid, et alustada._

---



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
